**Корректность проверена на Python 3.6:**
+ pandas 0.23.4
+ numpy 1.15.4
+ sklearn 0.20.2

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Sklearn

## sklearn.grid_search

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [2]:
from sklearn import model_selection, datasets, linear_model, metrics

import numpy as np
import pandas as pd

### Генерация датасета

In [3]:
iris = datasets.load_iris()

In [4]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                     test_size = 0.3,random_state = 0)

### Задание модели

In [5]:
classifier = linear_model.SGDClassifier(random_state = 0, tol=1e-3)

### Генерация сетки

In [6]:
classifier.get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [7]:
parameters_grid = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    'max_iter' : np.arange(5,10),
    'alpha' : np.linspace(0.0001, 0.001, num = 5),
}

In [8]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size = 0.2, random_state = 0)

### Подбор параметров и оценка качества

#### Grid search

In [9]:
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

In [14]:
%%time
grid_cv.fit(train_data, train_labels)

Wall time: 2.74 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             error_score='raise-deprecating',
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight=None, early_stopping=False,
                                     epsilon=0.1, eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='hinge', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=...
                                     shuffle=True, tol=0.001,
                                     validation_fraction=0.1, verbose=0,
                                     warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'loss': ['hinge', 'log', 'squared_hinge',
                 

In [11]:
grid_cv.best_estimator_

SGDClassifier(alpha=0.0007750000000000001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=9,
              n_iter_no_change=5, n_jobs=None, penalty='l1', power_t=0.5,
              random_state=0, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [35]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

0.9047619047619048
{'alpha': 0.000325, 'loss': 'squared_hinge', 'max_iter': 9, 'penalty': 'l1'}


In [15]:
grid_cv.cv_results_

{'mean_fit_time': array([0.00124276, 0.00114787, 0.00094092, 0.00089266, 0.00069439,
        0.0007437 , 0.00074389, 0.00079424, 0.00079334, 0.00074308,
        0.00079343, 0.00069416, 0.00084312, 0.00079355, 0.00089271,
        0.00074375, 0.00079415, 0.00084298, 0.00094259, 0.00089278,
        0.00079415, 0.00074315, 0.00084255, 0.00074315, 0.00079434,
        0.0006943 , 0.00089197, 0.0008431 , 0.00079339, 0.00074346,
        0.00074308, 0.00079303, 0.00084336, 0.00079372, 0.00089295,
        0.00074422, 0.00089605, 0.00079353, 0.00084295, 0.00084321,
        0.00074394, 0.00084343, 0.00079379, 0.00069447, 0.00084336,
        0.00079634, 0.00084288, 0.00084269, 0.00074806, 0.0007915 ,
        0.00084658, 0.00089595, 0.00084364, 0.00084274, 0.00089369,
        0.00089493, 0.00088897, 0.00089252, 0.00084243, 0.000892  ,
        0.00073998, 0.00089242, 0.00074332, 0.00074394, 0.0007935 ,
        0.00069427, 0.00079358, 0.00074394, 0.00084307, 0.00079341,
        0.00079343, 0.00079346,

#### Randomized grid search

In [16]:
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv, n_iter = 20, 
                                                   random_state = 0)

In [17]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

Wall time: 289 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
                   error_score='raise-deprecating',
                   estimator=SGDClassifier(alpha=0.0001, average=False,
                                           class_weight=None,
                                           early_stopping=False, epsilon=0.1,
                                           eta0=0.0, fit_intercept=True,
                                           l1_ratio=0.15,
                                           learning_rate='optimal',
                                           loss='hinge', max_iter=1000,
                                           n_iter_no_change=5, n...
                                           tol=0.001, validation_fraction=0.1,
                                           verbose=0, warm_start=False),
                   iid='warn', n_iter=20, n_jobs=None,
                   param_distributions={'alpha': array([0.0001  , 0.000

In [18]:
print(randomized_grid_cv.best_score_)
print(randomized_grid_cv.best_params_)

0.8666666666666667
{'penalty': 'l1', 'max_iter': 8, 'loss': 'squared_hinge', 'alpha': 0.0007750000000000001}


In [19]:
#!/usr/bin/python

from datetime import datetime


def control_sum(str):
    return sum(map(ord, list(str)))


def check(reply):
    def _is_number(str):
        try:
            int(str)
            return True
        except ValueError:
            return False

    if "Control sum:" not in reply:
        return False
    parts = reply.split("Control sum:")
    received_current_time = parts[0].strip()
    received_control_sum = parts[1].strip()
    if not _is_number(received_control_sum):
        return False
    else:
        received_control_sum = int(received_control_sum)
    expected_control_sum = control_sum(received_current_time)
    return expected_control_sum == received_control_sum


current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(current_time)
print("Control sum: " + str(control_sum(current_time)))

#print check(current_time + '\n' + "Control sum: " + str(control_sum(current_time)))

2019-12-13 16:39:21
Control sum: 951
